In [ ]:
!pip install wradlib

# **Fetching data from raw radar Data**

In [ ]:
import wradlib as wrl
import pyproj
import math
import warnings
import numpy as np

warnings.filterwarnings('ignore')

pi = math.pi
pi2 = pi / 180

# File path to the radar data file
fpath = 'PTL240604095222.RAWWZFZ'

# Read the radar data file using wradlib
fcontent = wrl.io.read_iris(fpath, loaddata=True, rawdata=False, debug=False)

my_list = [0.2, 1.0, 2.0, 3.0, 4.5, 6.0, 9.0, 12.0, 16.0, 21.0]

# Radar location (longitude, latitude, altitude)
radar_alt = fcontent['ingest_header']['ingest_configuration']['altitude_radar']
radar_lon = fcontent['ingest_header']['ingest_configuration']['longitude_radar']
radar_lat = fcontent['ingest_header']['ingest_configuration']['latitude_radar']

# Set up pyproj transformer
geod = pyproj.Geod(ellps="WGS84")
proj_aeqd = pyproj.Proj(proj='aeqd', lat_0=radar_lat, lon_0=radar_lon, datum='WGS84')
# print(fcontent['data'][1]['sweep_data'])
for i in range(1, 11):
    with open(f'polar40_{i}_1.txt', 'w') as g1:
        i1 = i - 1
        el = my_list[i1]
        ele = math.radians(my_list[i1])
        for j in range(360):
            tdata = fcontent['data'][i]['sweep_data']['DB_DBT2']
            j1 = math.radians(j)
            for k in range(501):
                z1 = tdata[j][k]
                if z1 < 25:
                    z1 = 0
                else:
                    zflag = 1
                    if z1 >= 30:
                        zflag = 2
                    if z1 >= 35:
                        zflag = 3
                    if z1 >= 40:
                        zflag = 4
                    if z1 >= 45:
                        zflag = 5
                    if z1 >= 50:
                        zflag = 6
                    if z1 >= 55:
                        zflag = 7
                    if z1 >= 60:
                        zflag = 8

                    elex = k * (math.cos(ele)) * (math.sin(j1))
                    eley = k * (math.cos(ele)) * (math.cos(j1))
                    elez = k * math.sin(ele)

                    lat = elex / 111.32 + radar_lat
                    lon = eley / (111.32 * np.cos(np.radians(radar_lat))) + radar_lon
                    # print(lat,lon)
                    alt = elez * 1000  # Assuming elevation in meters)
                    # Append the line to the file
                    g1.write(f"{k} \t {j} \t {el} \t {elex} \t {eley} \t {elez} \t {z1} \t {zflag} \t {lat} \t {lon} \t {alt}\n")

# **Clustering and 3D rendering with Interactive plots**

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
from collections import defaultdict
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull, QhullError

def dbscans(a):
    if a.size == 0:
        return {}
    if len(np.shape(a)) == 1:
        a = np.reshape(a, (1, 11))
    aY = a[:, 0:2]
    aZ = a[:, 0:11]
    adb = DBSCAN(eps=1, min_samples=3).fit(aY)
    alab = adb.labels_
    dic = {label: aZ[alab == label] for label in np.unique(alab)}
    return dic

def load_data(elevations):
    data = {}
    for elevation in elevations:
        data[elevation] = np.loadtxt(f'polar40_{elevation}_1.txt')
    return data

def calculate_overlap(cluster1, cluster2):
    set1 = set(map(tuple, cluster1[:, 0:2]))
    set2 = set(map(tuple, cluster2[:, 0:2]))
    intersection = set1 & set2
    overlap_percentage1 = len(intersection) / len(set1)
    overlap_percentage2 = len(intersection) / len(set2)
    return overlap_percentage1, overlap_percentage2

def find_intersections(cluster_data):
    intersections = defaultdict(list)
    for i in range(1, 9):
        for j in range(i + 1, 9):
            for label_i, points_i in cluster_data[i].items():
                for label_j, points_j in cluster_data[j].items():
                    overlap_percentage1, overlap_percentage2 = calculate_overlap(points_i, points_j)
                    if (overlap_percentage1 >= 0.05) and (overlap_percentage2 >= 0.05):
                        intersections[(i, label_i)].append((j, label_j))
                        intersections[(j, label_j)].append((i, label_i))
    return intersections

def group_clusters(intersections):
    grouped_clusters = defaultdict(set)
    visited = set()

    def dfs(label):
        stack = [label]
        group = set()
        while stack:
            current = stack.pop()
            if current in visited:
                continue
            visited.add(current)
            group.add(current)
            for neighbor in intersections[current]:
                if neighbor not in visited:
                    stack.append(neighbor)
        return group

    for label in intersections:
        if label not in visited:
            group = dfs(label)
            grouped_clusters[frozenset(group)] = group

    return grouped_clusters

def plot_clusters(cluster_data, grouped_clusters):
    fig = go.Figure()
    plt.figure()
    group_counter = 1

    for group in grouped_clusters:
        group_points = []

        for (elevation, label) in grouped_clusters[group]:
            points = cluster_data[elevation][label][:, 8:10]
            group_points.append(points)
            fig.add_trace(go.Scatter(
                x=points[:, 0],
                y=points[:, 1],
                mode='markers',
                marker=dict(
                    size=3,
                    color=cluster_data[elevation][label][:, 7],  # Color by seventh column
                    colorscale='Viridis',  # Choose a colorscale for coloring
                    colorbar=dict(title='Reflectivity')
                ),
                name=f'Group {group_counter} (Elevation {elevation} Cluster {label})'
            ))

        group_points = np.vstack(group_points)

        if len(group_points) > 3:  # Ensure there are enough points to form a hull
            try:
                hull = ConvexHull(group_points)
                hull_points = group_points[hull.vertices]

                plt.plot(group_points[:, 0], group_points[:, 1], 'o')
                plt.plot(hull_points[:, 0], hull_points[:, 1], 'r--', lw=2)
                plt.fill(hull_points[:, 0], hull_points[:,1], 'r', alpha=0.3)

                # Create a Polygon patch for the hull
                poly = Polygon(hull_points, closed=True, fill=None, edgecolor='r')
                plt.gca().add_patch(poly)
            except QhullError:
                print(f"Warning: Unable to create ConvexHull for group {group}. Points may be collinear or insufficient.")

        group_counter += 1

    layout = {
        'xaxis': {'title': 'lat'},
        'yaxis': {'title': 'long'},
        'title': 'Grouped Clusters'
    }
    fig.update_layout(layout)
    fig.show()
    plt.xlabel('lat')
    plt.ylabel('long')
    plt.title('Grouped Clusters with Convex Hulls')
    plt.show()

def plot_clusters_3d(cluster_data, grouped_clusters):
    fig = go.Figure()

    group_counter = 1

    for group in grouped_clusters:
        group_points = []
        for (elevation, label) in grouped_clusters[group]:
            points = cluster_data[elevation][label][:, 3:6]  # Use columns 8, 9, 10 for 3D plotting
            group_points.append(points)

            # Add scatter trace for each cluster
            fig.add_trace(go.Scatter3d(
                x=points[:, 0],
                y=points[:, 1],
                z=points[:, 2],
                mode='markers',
                marker=dict(
                    size=5,
                    color=cluster_data[elevation][label][:, 7],  # Color by seventh column
                    colorscale='Viridis',  # Choose a colorscale for coloring
                    colorbar=dict(title='Column 7')
                ),
                name=f''
            ))

        group_counter += 1

    # Update layout
    layout = {
        'scene': {
            'xaxis': {'title': 'X'},
            'yaxis': {'title': 'Y'},
            'zaxis': {'title': 'Z'}
        },
        'title': 'Grouped Clusters in 3D'
    }
    fig.update_layout(layout)

    fig.show()

def plot_individual_group(cluster_data,grouped_clusters,idx, xlim=None, ylim=None, zlim=None):
    fig = go.Figure()

    group_points = []

    for (elevation, label) in grouped_clusters[idx]:
        points = cluster_data[elevation][label][:, 3:6]  # Use columns 8, 9, 10 for 3D plotting
        group_points.append(points)

        # Add scatter trace for each cluster
        fig.add_trace(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(
                size=5,
                color=cluster_data[elevation][label][:, 7],  # Color by seventh column
                colorscale='Viridis',  # Choose a colorscale for coloring
                colorbar=dict(title='Column 7')
            ),
            name=f''
        ))

    # Update layout
    layout = {
        'scene': {
            'xaxis': {'title': 'X', 'range': xlim},
            'yaxis': {'title': 'Y', 'range': ylim},
            'zaxis': {'title': 'Z', 'range': zlim}
        },
        'title': f'Individual Group in 3D'
    }
    fig.update_layout(layout)

    fig.show()

def checks(dat, cri):
    aa = np.where(dat[:, 6] > cri)
    aaa = dat[aa[0], :]
    if len(aaa) >= 4:
        dic2 = dbscans(aaa)
        return dic2
    else:
        return 'no'

def driver_function(data):
    dic1 = dbscans(data)
    datas = {}
    for dd1 in dic1:
        if dd1 >= 0:
            dlist = [dic1[dd1]]
            dkey = [25]
            unique_cluster_found = False
            ii = 0

            while ii < len(dlist):
                dic2 = checks(dlist[ii], dkey[ii])
                if dic2 != 'no':
                    unique_labels = [dd2 for dd2 in dic2 if dd2 >= 0]
                    if len(unique_labels) == 1:  # Check if there's only one unique cluster
                        unique_cluster_found = True
                        dlist.append(dic2[unique_labels[0]])
                        dkey.append(dkey[ii] + 5)
                    else:
                        if unique_cluster_found:
                            datas[str(dd1) + '_' + str(ii) + '_' + str(dkey[ii])] = dlist[ii]
                        break
                else:
                    datas[str(dd1) + '_' + str(ii) + '_' + str(dkey[ii])] = dlist[ii]
                    break
                ii += 1

    return datas


def main():
    elevations = [1, 2, 3, 4, 5, 6, 7, 8]
    clustered_data = {}

    for elevation in elevations:
        data = np.loadtxt(f'polar40_{elevation}_1.txt')
        clustered_data[elevation] = driver_function(data)

    all_cluster_data = {}
    for elevation, clusters in clustered_data.items():
        for cluster_label, cluster_points in clusters.items():
            all_cluster_data[(elevation, cluster_label)] = cluster_points

    intersections = find_intersections(clustered_data)
    grouped_clusters = group_clusters(intersections)
    plot_clusters(clustered_data, grouped_clusters)

    # Plot all grouped clusters
    plot_clusters_3d(clustered_data, grouped_clusters)

    # Example: Plotting individual group 1 (adjust xlim, ylim, zlim as needed)
    i=1
    for idx,group in enumerate(grouped_clusters):
      if idx==7:
        plot_individual_group(clustered_data, grouped_clusters,group, xlim=(-100, 500), ylim=(0, 500), zlim=(0, 20))

main()